# Descrição inicial
blabla isso classifica caes e gatinhos
# sumario pra acompanhar?


## Instalando dependências

In [ ]:
!pip install LTNtorch kagglehub pandas

## Importando bibliotecas

In [ ]:
import pandas as pd
import os
import warnings
import kagglehub
from kagglehub import KaggleDatasetAdapter

## Baixando dataset com imagens de cães e gatos

In [ ]:
dataset_name = "shaunthesheep/microsoft-catsvsdogs-dataset"
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    dataset_name
)

print("Dataset importado com sucesso")
print(df.head())